# Setting up the *p-Arm* configuration

Through this guide the basic params for the *p-Arm* manipulator will be set-up.

The required options are:

+ The **end-effector** length.
+ The **lower arm** length.
+ The **upper arm** lenght.
+ The **base** height.
+ The **little deviation** from base.

All these information conforms de *Denavit-Hartenberg* table which allows us to understand how the movement of each motor (joint) will affect the final position. For this arm in particular, it is working on the $XZ$ axis, so the first $a_i$ is swapped with the correspondant $d_i$.

In addition, we have to know the angle in between each joint. Having a look to the picture, we know that only the first segment and the second one have an angle different to '0'.

![uArm sizes and configuration](https://github.com/UPM-Robotics/uarm/raw/master/docs/memory/images/sizes.png "uArm sizes")

Finally, as the *end-effector* position is always parallel to the floor (this means its angle is always $π$), we can remove it from the table and include it as a traslation in the '$Z$' axis.

So then, the final table results as:

| $i$ | $θ_i$ | $d_i (mm.)$ | $a_i (mm.)$ | $α_i$ |
|---|:--:|----------|----------|----|
| $1$ | $θ_1$ |       $d_1$ |       $a_1$ | $α_1$ |
| $2$ | $θ_2$ |        $0$ |       $a_2$ | $α_2$ |
| $3$ | $θ_3$ |        $0$ |       $a_3$ |  $0$ |

## Install the required packages

For running, it is necessary to install the `sympy` module (see [sympy on GitHub](https://github.com/sympy/sympy)) and `numpy` ([numpy](https://numpy.org/)).

In [ ]:
pip install sympy numpy

Now, import `DHTable` from the `manipulator` package and start defining things.

As we saw in the previous table, we need three symbols: $θ_1$, $θ_2$, $θ_3$.

In [ ]:
from manipulator import (
    DHTable,
    pi,
    Manipulator
)
from sympy import symbols

parm_table = DHTable()
t1, t2, t3 = symbols("theta_1 theta_2 theta_3")

Now, as we have our three symbols, we must fill the other values necessary to the table to work properly. As shown in the table, we need to define:

+ $a_1$, $a_2$, $a_3$.
+ $d_1$.
+ $α_1$, $α_2$.
+ $T_X$, $T_Z$.

The latest ones ($T_x$, $T_z$) stand for the traslation in the $X$ axis and traslation in $Z$ axis.

In [ ]:
# Different arm lengths
## For specific lengths
# a1 = 13.2
# a2 = 142.07
# a3 = 158.81
## For symbolic calculus
a1, a2, a3 = symbols("a_1 a_2 a_3")

# The deviation present in between joints
## For specific lengths
# d1 = 106.6
## For symbolic calculus
d1 = symbols("d_1")

# The traslations in both X and Z axis
## For specific lengths
# Tx = 44.5
# Tz = -13.2
## For symbolic calculus
Tx, Tz = symbols("T_X T_Z")

# The angle in between joints
alpha_1 = (pi / 2)
alpha_2 = pi

Then, we can define the `DHTable` by using the `add` method, as follows:

In [ ]:
## False for symbolic calculus
is_digits = False
parm_table.add(theta=t1, d=d1, a=a1, alpha=alpha_1, check_attrs=is_digits)\
          .add(theta=t2, d=0, a=a2, alpha=alpha_2, check_attrs=is_digits)\
          .add(theta=t3, d=0, a=a3, alpha=0, check_attrs=is_digits)
parm_table.Tx = Tx
parm_table.Tz = Tz

The generated table is:

In [ ]:
print(parm_table)

Now, we can declare a `Manipulator` with the obtained data, so we can directly obtain the forward kinematics matrices:

In [ ]:
from pprint import pprint
from IPython.display import display, Latex

pArm = Manipulator(params=parm_table)

latmat_tmpl = "\\begin{{equation*}}^{0}T_{1}={2}\\end{{equation*}}"
# print each matrix data
for matrix in ['A01', 'A12', 'A23', 'A02', 'A03']:
    mat = latmat_tmpl.format(matrix[1], matrix[2], pArm.to_latrix('p', matrix))
    display(Latex(mat))

In this case, we are interested in the matrix which is the multiplication of all the other matrices, in this case `A03`

In [ ]:
pprint(pArm.direct_kinematics['A03'])

More beautifully:

In [ ]:
latmat = f"\\begin{{equation*}}^0T_3={pArm.to_latrix('p', 'A03')}\\end{{equation*}}"
display(Latex(latmat))

Obtaining for $X$, $Y$, $Z$:

In [ ]:
from sympy import latex

x = pArm.direct_kinematics['A03'][0, 3]
y = pArm.direct_kinematics['A03'][1, 3]
z = pArm.direct_kinematics['A03'][2, 3]

xyz_latex = \
f"""\\begin{{equation}}\n
    \\left.\\begin{{aligned}}\n
        X_e &= {latex(x)} \\\\\n
        Y_e &= {latex(y)} \\\\\n
        Z_e &= {latex(z)} \\\\\n
    \\end{{aligned}}\\right\\}}\n
\\end{{equation}}"""

display(Latex(xyz_latex))

## Obtaining the Jacobian matrices

The Jacobina matrices allows us to obtain the relationship in between joints angular speed
and end-effector linear speed. They are defined as the partial derivative of both $\dot{x}$ and $\dot{q}$, and it is defined as:

\begin{equation}
    J_{ee}\left(\dot{q}\right) = 
    \begin{bmatrix}
        J_v\left(\dot{q}\right) \\
        J_{\omega}\left(\dot{q}\right)
    \end{bmatrix}
\end{equation}

in which $J_v\left(\dot{q}\right)$ is defined by:

\begin{equation}
    J_v\left(\dot{q}\right) = 
    \begin{bmatrix}
        \frac{\partial X_e}{\partial q_0} & \frac{\partial X_e}{q_1} & \cdots & \frac{\partial X_e}{q_n} & \frac{\partial X_e}{d_1} & \cdots & \frac{\partial X_e}{d_n} \\[3ex]
        \frac{\partial Y_e}{\partial q_0} & \frac{\partial Y_e}{q_1} & \cdots & \frac{\partial Y_e}{q_n} & \frac{\partial Y_e}{d_1} & \cdots & \frac{\partial Y_e}{d_n} \\[3ex]
        \frac{\partial Z_e}{\partial q_0} & \frac{\partial Z_e}{q_1} & \cdots & \frac{\partial Z_e}{q_n} & \frac{\partial Z_e}{d_1} & \cdots & \frac{\partial Z_e}{d_n} \\
    \end{bmatrix}
\end{equation}

and $J_{\omega}\left(\dot{q}\right)$ is defined by:

\begin{equation}
    J_{\omega}\left(\dot{q}\right) =
    \begin{bmatrix}
        \frac{\partial \phi_X}{\partial q_0} & \frac{\partial \phi_X}{q_1} & \cdots & \frac{\partial \phi_X}{q_n} & \frac{\partial \phi_X}{d_1} & \cdots & \frac{\partial \phi_X}{d_n} \\[3ex]
        \frac{\partial \phi_Y}{\partial q_0} & \frac{\partial \phi_Y}{q_1} & \cdots & \frac{\partial \phi_Y}{q_n} & \frac{\partial \phi_Y}{d_1} & \cdots & \frac{\partial \phi_Y}{d_n} \\[3ex]
        \frac{\partial \phi_Z}{\partial q_0} & \frac{\partial \phi_Z}{q_1} & \cdots & \frac{\partial \phi_Z}{q_n} & \frac{\partial \phi_Z}{d_1} & \cdots & \frac{\partial \phi_Z}{d_n} \\
    \end{bmatrix}
\end{equation}

For the *p*Arm manipulator, we need to define the following rules:

\begin{align*}
    \phi_X &= \theta_2 - \theta_3 \\
    \phi_Y &= 0 \\
    \phi_Z &= \theta_1
\end{align*}

In [ ]:
pArm.set_phi('x', t2 - t3)
pArm.set_phi('y', 0)
pArm.set_phi('z', t1)

And we can obtain the Jacobian matrix:

In [ ]:
from manipulator.utils import to_latrix

j = pArm.jacobian(subs=(t1, t2, t3))
jacobian_latex = \
f"""
\\begin{{equation}}
    J_{{ee}}\\left(\\dot{{q}}\\right) = {to_latrix('b', j)}
\\end{{equation}}
"""
display(Latex(jacobian_latex))

from which we can obtain the submatrices:

In [ ]:
jv = j[0:3, 0:3]
jq = j[3:6, 0:3]

matrices = \
f"""
\\begin{{align}}
    J_v\\left(\\dot{{q}}\\right) &= {to_latrix('b', jv)} \\\\
    J_{{\\omega}}\\left(\\dot{{q}}\\right) &= {to_latrix('b', jq)}
\\end{{align}}
"""
display(Latex(matrices))

For obtaining the inverse Jacobian matrix, there are two ways for it:

+ By trying to obtain directly the inverse Jacobian, if the determinant is zero.
+ By obtaining the pseudo-inverse, which is defined by the following equation:

\begin{equation}
    J^+ = J^T (J \cdot J^T)^{-1}
\end{equation}

The pseudo-inverse always exists and, in case the inverse also exists, it equals the
inverse Jacobian:

\begin{equation}
    J^+ \equiv J^{-1} \iff \exists J^{-1}
\end{equation}

For the *p*Arm manipulator, the values obtained for the inverse Jacobian are the following:

In [ ]:
ij = pArm.inverse

ij_matrix = \
f"""
\\begin{{align}}
    J^{{-1}} &= {to_latrix('b', ij)} \\\\[2ex]
    \\left|J_{{ee}}\\left(\\dot{{q}}\\right)\\right| &= {latex(pArm.inverse_kinematics.det)}
\\end{{align}}
"""

display(Latex(ij_matrix))

In the equations below we can see that the determinant is zero iff $\theta_3 = 0$ or $\theta_3 = \pi$ but, due to the arm configuration, those angles are not possible as it is limited by the arm.